In [218]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import math
import time
import pyproj
import random
import cores as cr
#import rasterio
#from rasterio import features

In [219]:
Rastro = [0xff, 0xff, 0x00]
Produzindo =[0xff,0xff,0x00]
Ocioso = [0xd6,0x27,0x28]
Manobrando = [0xff,0x7e,0x0e]
Deslocando= [0x1f,0x76,0xb4]
App_H3 = [0x00, 0x8a,0x33]
App_H2 = [0x82,0xcc,0x00]
App_H1 = [0xcc,0xe2,0x00] 
App_0  = [0xff,0xee,0x00]
App_L1 = [0xff,0xaf,0x00]
App_L2 = [0xff,0x78,0x00]
App_L3 = [0xcc,0x00,0x00]

In [220]:
file = 'gpd.csv'

MERCATOR = pyproj.CRS('epsg:3857')
WGS84 = pyproj.CRS('epsg:4326')
SAD1969  = pyproj.CRS('esri:102033')

df = pd.read_csv(file)

df = df.loc[(df.cor != 'humidity_rastro') & (df.cor != 'produzindo_rastro')]
df.cor = df.cor.str.capitalize()

"""  """

df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')

application_color_list = ['App_L3', 'App_L2', 'App_L1', 'App_0', 'App_H1', 'App_H2', 'App_H3']
camada_objeto = 'App_rastro'
if len(gdf[gdf['cor'] == camada_objeto]['value'].dropna().unique()) > 1:
    print("Tiles - processar polígonos de velocidade")
    inicial = gdf[gdf['cor'] == camada_objeto]['value'].dropna().quantile(0.1)
    final = gdf[gdf['cor'] == camada_objeto]['value'].dropna().quantile(0.9)
    topo = gdf[gdf['cor'] == camada_objeto]['value'].dropna().quantile(1)
    n_intervalos = 7
    passo = (final - inicial) / (n_intervalos - 2)
    atual = inicial
    intervalos = []
    intervalos.append([0, inicial])
    for i in list(range(2, n_intervalos)):
        intervalos.append([atual, atual + passo])
        atual = atual + passo
    intervalos.append([final, topo + 1])
    pos_int = 0
    for intervalo in intervalos:
        gdf.loc[(gdf['cor'] == camada_objeto) & (gdf['value'] >= intervalo[0]) & (gdf['value'] < intervalo[1]), 'cor'] = application_color_list[pos_int]
        pos_int = pos_int + 1
gdf = gdf.reset_index(drop=True)
source = ogr.Open(gdf.to_json())

gdf


Tiles - processar polígonos de velocidade


,Unnamed: 0,geometry,cor,application_rate,value,cultural_practice,bin_element_numbers,start_date,end_date,type
0,0,"LINESTRING (-89.77969 40.02795, -89.77969 40.0...",Produzindo,NaN,6.661111,0,0,2022-09-28 23:15:07.700,2022-09-28 23:15:43.700,RASTRO
1,1,"LINESTRING (-89.77961 40.02747, -89.77961 40.0...",Produzindo,NaN,6.302778,0,0,2022-09-28 23:16:04.700,2022-09-28 23:16:41.700,RASTRO
2,2,"LINESTRING (-89.77963 40.02807, -89.77963 40.0...",Produzindo,NaN,7.022764,0,0,2022-09-28 23:16:43.700,2022-09-28 23:18:46.700,RASTRO
3,3,"LINESTRING (-89.77959 40.03013, -89.77959 40.0...",Produzindo,NaN,6.549242,0,0,2022-09-28 23:21:15.700,2022-09-28 23:23:27.700,RASTRO
4,4,"LINESTRING (-89.77955 40.02793, -89.77955 40.0...",Produzindo,NaN,7.026667,0,0,2022-09-28 23:23:31.700,2022-09-28 23:24:01.700,RASTRO
...,...,...,...,...,...,...,...,...,...,...
187,337,"POLYGON ((-89.77926 40.02972, -89.77926 40.029...",App_L1,NaN,1.401465,0,0,2022-09-28 23:41:48.000,2022-09-28 23:41:56.000,APPLICATION
188,338,"POLYGON ((-89.77926 40.02986, -89.77926 40.029...",App_L1,NaN,1.468777,0,0,2022-09-28 23:41:56.000,2022-09-28 23:42:04.000,APPLICATION
189,339,"POLYGON ((-89.77925 40.03000, -89.77918 40.029...",App_H3,NaN,2.774431,0,0,2022-09-28 23:42:04.000,2022-09-28 23:42:06.000,APPLICATION
190,340,"POLYGON ((-89.77912 40.02991, -89.77912 40.029...",App_L3,NaN,0.077542,0,0,2022-09-28 23:44:50.000,2022-09-28 23:45:01.000,APPLICATION


In [221]:
pd.unique(gdf.cor)

array(['Produzindo', 'Ocioso', 'Deslocando', 'App_L1', 'App_L2', 'App_0',
       'App_H3', 'App_H2', 'App_H1', 'App_L3'], dtype=object)

In [222]:
source_ds = ogr.GetDriverByName("Memory").CopyDataSource(source, "")
source_layer = source_ds.GetLayer(0)
source_srs = source_layer.GetSpatialRef()
x_min, x_max, y_min, y_max = source_layer.GetExtent()
numFeatures = source_layer.GetFeatureCount()


#definindo field de cor vermelha
source_layer.CreateField(ogr.FieldDefn('Red', ogr.OFTReal))
source_layer_def = source_layer.GetLayerDefn()
r_id = source_layer_def.GetFieldIndex('Red')

#definindo field de cor verde
source_layer.CreateField(ogr.FieldDefn('Green', ogr.OFTReal))
source_layer_def = source_layer.GetLayerDefn()
g_id = source_layer_def.GetFieldIndex('Green')

#definindo field de cor azul
source_layer.CreateField(ogr.FieldDefn('Blue', ogr.OFTReal))
source_layer_def = source_layer.GetLayerDefn()
b_id = source_layer_def.GetFieldIndex('Blue')


fd_id = source_layer_def.GetFieldIndex('cor')
ldefn = source_layer.GetLayerDefn()
ldefn.GetFieldCount()
lsAtribNames = [field.name for field in source_layer.schema]

In [223]:
lsAtribNames

['id',
 'Unnamed: 0',
 'cor',
 'application_rate',
 'value',
 'cultural_practice',
 'bin_element_numbers',
 'start_date',
 'end_date',
 'type',
 'Red',
 'Green',
 'Blue']

In [224]:
for feature in source_layer:
    feature.SetField(r_id,globals()[feature.GetField(fd_id)][0])
    source_layer.SetFeature(feature)

In [225]:
for feature in source_layer:
    feature.SetField(g_id,globals()[feature.GetField(fd_id)][1])
    source_layer.SetFeature(feature)

In [226]:
for feature in source_layer:
    feature.SetField(b_id,globals()[feature.GetField(fd_id)][2])
    source_layer.SetFeature(feature)

In [227]:
i = 191
globals()[source_layer[i].GetField(fd_id)],source_layer[i].GetField(r_id),source_layer[i].GetField(g_id),source_layer[i].GetField(b_id)

([204, 0, 0], 204.0, 0.0, 0.0)

In [228]:
pix_size = 0.000002
x_res = int((x_max - x_min) / pix_size)
y_res = int((y_max - y_min) / pix_size)

In [229]:
target_ds = gdal.GetDriverByName('GTiff').Create('rasterize1.tif', x_res, y_res, 4, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pix_size, 0, y_max, 0, -pix_size))

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'

if source_srs:
    target_ds.SetProjection(source_srs.ExportToWkt())
else:
    target_ds.SetProjection(proj)

#err = gdal.RasterizeLayer(target_ds, [1,2,3], source_layer, burn_values=(0xff, 0xff, 0xff))

err = gdal.RasterizeLayer(target_ds, [1], source_layer, options=["ATTRIBUTE=%s" % 'Red'])
err = gdal.RasterizeLayer(target_ds, [2], source_layer, options=["ATTRIBUTE=%s" % 'Green'])
err = gdal.RasterizeLayer(target_ds, [3], source_layer, options=["ATTRIBUTE=%s" % 'Blue'])
err = gdal.RasterizeLayer(target_ds, [4], source_layer, burn_values=[0xff])



pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize1.png', target_ds)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001602E7BEC10> >

In [230]:
x_size = gdf.total_bounds[2]-gdf.total_bounds[0]
y_size = gdf.total_bounds[3]-gdf.total_bounds[1]
x_0 = gdf.total_bounds[0]
y_0 = gdf.total_bounds[3]

 

pix_size = 0.000002
pixels_in_x = math.ceil(x_size/pix_size)
pixels_in_y = math.ceil(y_size/pix_size)

 
sr_wkt = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
sr = osr.SpatialReference( sr_wkt )


# Create a memory raster to rasterize into.

 

target_ds = gdal.GetDriverByName('MEM').Create('', pixels_in_x, pixels_in_y, 4, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_0, pix_size, 0, y_0, 0, -pix_size))

 

# Create a memory layer to rasterize from.

 
rast_ogr_ds  = ogr.GetDriverByName('Memory').CreateDataSource('wrk')
rast_mem_lyr = rast_ogr_ds.CreateLayer('poly', srs=sr)


for g in gdf[gdf['cor'] == 'produzindo_rastro'].geometry:
    feat = ogr.Feature(rast_mem_lyr.GetLayerDefn())
    feat.SetGeometryDirectly(ogr.Geometry(wkt = g.wkt))
    rast_mem_lyr.CreateFeature(feat)

gdal.RasterizeLayer(target_ds, [4,3,2,1], rast_mem_lyr, burn_values=(0, 0xff, 0xff, 0 ))

""" , options = ["BURN_VALUE_FROM="]   ,  ,options=['ATTRIBUTE=values']  burn_values=(0xFFFF, 0xFFFF, 0xFFFF, 0 )"""

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
target_ds.SetProjection(proj)

 

tiffdriver = gdal.GetDriverByName('GTiff')
tiffdriver.CreateCopy('rasterize.tif', target_ds)

 

pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize.png', target_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001602E6B6BB0> >